# Double stained (PD-L1, CD163)
## Segmentation of Macrophages (CD163 - Red) via nuclei
See Notebook 07-299.1.DOUBLE-Segmentation_on_Red_dye_CD163 for initial analysis

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np

In [ ]:
# Set up matplotlib defaults: larger images, gray color map
import matplotlib
matplotlib.rcParams.update({
    'figure.figsize': (10, 10),
    'image.cmap': 'gray'
})

In [ ]:
import skimage as sk
from skimage import data, io
from skimage import segmentation, morphology, color, feature
from skimage import filters
from scipy import ndimage as ndi
from skimage import util
from skimage import exposure
from skimage.color import label2rgb
from skimage.color import separate_stains
import os

In [ ]:
# Segement of slide 07-299.1.DOUBLE
# Double stained: Red (CD163, macrophages), DAB/Brown (PD-L1)
# Hematoxylin stained nuclei (weak) - blue/purple
slide = io.imread('../Images/Double_PD-L1_CD163/07-299.1.DOUBLE (1, x=76249, y=78898, w=2216, h=1510).tif')
plt.imshow(slide);

### Preprocessing
#### Calculate dye vector manully from red pixel values
Take median values for each channel of range of red pixels - wide range of shades/intensity

In [ ]:
def get_dye_vector(a):
    """
    Function that takes rgb values as a list.
    Returns normalised OD (dye vector).
    """
    a = np.array([a], dtype=np.uint8)
    a_float = sk.img_as_float(a)
    OD = -np.log10(a_float)
    normalised_OD = OD / np.sqrt(np.sum(OD**2))
    dye_vector = np.around(normalised_OD, decimals=3)
    return dye_vector[0]

In [ ]:
rgb_red_range = np.array([
                  [205, 105, 129],
                  [218, 109, 132],
                  [201, 109, 130],
                  [209, 102, 128],
                  [203, 115, 140],
                  [211, 107, 132],
                  [206, 106, 130],
                  [219, 102, 128],
                  [205, 139, 139],
                  [229, 105, 129]])

rgb_red_med = np.median(rgb_red_range, axis=0)
print(rgb_red_med)

In [ ]:
red_dye_vector = get_dye_vector(rgb_red_med)
print(red_dye_vector)

## 1. Colour Deconvolution using separate_stains and convolution matrix for H, DAB, Red (hdr)

In [ ]:
# Convolution Matrix
# channel 1 [0] = Hematoxylin (h)
# channel 2 [1] = DAB/Brown (PD-L1) (d)
# channel 3 [2] = Red dye (CD163) (r)

# Deconvolute into grayscale images of the three dyes (hdr)

# Values for rgb_from_hdr_default matrix taken from: 
# 1. H and DAB default stain vectors (QuPath)
# 2. Red dye caluculated manually from range of red pixels using 07-299.1 (cells above above)
# Code defines ‘rgb’ values from a combination of the H, Red and DAB dyes - ‘dye vectors’
rgb_from_hdr_default = np.array([[0.651, 0.701, 0.29], [ 0.269, 0.568, 0.778], red_dye_vector])

#Inverse matirx (convolution matrix) - going from rgb values to hdr values
hdr_from_rgb_default = np.linalg.inv(rgb_from_hdr_default)

deconvolute_default = separate_stains(slide, hdr_from_rgb_default)

#def separate_stains(rgb, conv_matrix):
#    rgb = _prepare_colorarray(rgb, force_copy=True)
#    rgb += 2
#    stains = np.reshape(-np.log10(rgb), (-1, 3)) @ conv_matrix [Reshape image matrix to get 3 rgb columns and convert to OD. Multiply by convolution matrix.]
#    return np.reshape(stains, rgb.shape)


In [ ]:
# This is a utility function that we'll use for display in a while;
# Taken from: Image Analysis in Python with SciPy and Scikit Image | Scipy 2019 Tutorial | Nunez-Iglesias 

def shuffle_labels(labels):
    """Shuffle the labels so that they are no longer in order.
    This helps with visualization.
    """
    indices = np.unique(labels[labels != 0])
    indices = np.append(
        [0],
        np.random.permutation(indices)
    )
    return indices[labels]

In [ ]:
def plot_figs(images, titles, shuffle=False):
    
    """
    Takes list of images and list of titles.
    Returns grid of images.
    """
    cols = 2
    if len(images) % 2 == 0:
        rows = int(len(images)/2)
    else:
        rows = int((len(images)/2) + 1)
    
    fig, axes = plt.subplots(rows, cols, figsize=(10, 10), sharex=True, sharey=True)
    ax = axes.ravel()
    
    for i in range(len(images)):
        if not shuffle:
            ax[i].imshow(images[i])
            ax[i].set_title(titles[i])
        elif shuffle:
            ax[i].imshow(shuffle_labels(images[i]), cmap='magma')
            ax[i].set_title(titles[i])
        
    for a in ax.ravel():
        a.axis('off')
        
    fig.tight_layout()

In [ ]:
# Plot in grayscale (dye deconvolution use default H and DAB dye vectors)
images_deconv_default = [slide, deconvolute_default[:, :, 0], deconvolute_default[:, :, 2], deconvolute_default[:, :, 1]]
titles_deconv_default = ["Original image", "Hematoxylin (default)", "Red - CD163", "DAB (default)"]
plot_figs(images_deconv_default, titles_deconv_default)

## Red dye channel

#### Tutorial:
Image Analysis in Python with SciPy and Scikit Image | Scipy 2019 Tutorial | Nunez-Iglesias
https://www.youtube.com/watch?v=d1CIV9irQAY

## 2. Filters and Thresholds

In [ ]:
# Grayscale image - Red dye channel

red = deconvolute_default[:, :, 2]
plt.imshow(red);

In [ ]:
# Smoothing - filter (remove noise)

red_smooth = ndi.median_filter(util.img_as_float(red), size = 5)
plt.imshow(red_smooth);

In [ ]:
red_smooth.dtype

In [ ]:
# improve contrast
red_gamma = exposure.adjust_gamma(red_smooth, 0.7)
plt.imshow(red_gamma);

In [ ]:
# Select Threshold for segmentation:
# Using Ostu Threshold as a start for analysis

otsu_threshold = filters.threshold_otsu(red_gamma)
red_otsu = red_gamma > otsu_threshold
print("Otsu Threshold = ", otsu_threshold)

In [ ]:
images_threshold = [slide, red_gamma, red_otsu]
titles_threshold = ["Original image", "Red - smooth, gamma", "Red - Otsu"]

plot_figs(images_threshold, titles_threshold)

In [ ]:
# remove small objects

red_otsu_rm = morphology.remove_small_objects(red_otsu, min_size=500)
plt.imshow(red_otsu_rm);

In [ ]:
# Fill in holes/nuclei
red_fill = ndi.morphology.binary_fill_holes(red_otsu_rm)
plt.imshow(red_fill);

In [ ]:
red_erode = morphology.binary_erosion(red_fill)
plt.imshow(red_erode);

## Hematoxylin - nuclei

In [ ]:
h = deconvolute_default[:, :, 0]
plt.imshow(h);

In [ ]:
# Smoothing - filter (remove noise)
h_smooth = ndi.median_filter(util.img_as_float(h), size = 5)
plt.imshow(h_smooth);

In [ ]:
h_otsu_threshold = filters.threshold_otsu(h_smooth)
h_otsu = h_smooth > h_otsu_threshold
print("Otsu Threshold = ", h_otsu_threshold)

In [ ]:
plt.imshow(h_otsu);

In [ ]:
h_otsu_rm = morphology.remove_small_objects(h_otsu)
plt.imshow(h_otsu_rm);

In [ ]:
h_fill = ndi.morphology.binary_fill_holes(h_otsu_rm)
plt.imshow(h_fill);

In [ ]:
h_erode = morphology.binary_erosion(h_fill)
plt.imshow(h_erode);

In [ ]:
h_erode2 = morphology.erosion(h_fill)
plt.imshow(h_erode2);

In [ ]:
red_nuclei = (h_erode & red_erode)
plt.imshow(red_nuclei);

In [ ]:
red_nuclei_rm = morphology.remove_small_objects(red_nuclei, min_size=20)
plt.imshow(red_nuclei_rm);

In [ ]:
images_nuclei = [slide, h_erode, red_erode, red_nuclei_rm]
titles_nuclei = ["Original image", "Nuclei", "Red Mask", "Red nuclei"]

plot_figs(images_nuclei, titles_nuclei)

## 3. Watershed Segmentation
### Count nucleated macrophages only - use hematoxylin stained nuclei to segment macrophages
Red dye channel (CD163) to get mask - Macrophages (red stained cells within tissue sample)
<br> Red dye channel AND Hematoxylin channel - segment by nuclei

In [ ]:
distance_red_nuclei = ndi.distance_transform_edt(red_nuclei_rm) 
#Distance map - distance to travel to background
#Smoother mask
plt.imshow(exposure.adjust_gamma(distance_red_nuclei, 0.5))
plt.title('Distance to background map: Red nuclei');

In [ ]:
local_peak_nuclei = feature.peak_local_max(distance_red_nuclei, min_distance=25, indices=False)
local_peak_nuclei.shape

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))

maxi_coords_nuclei = np.nonzero(local_peak_nuclei)
# np.nonzero - Return the indices of the elements that are non-zero.
ax.imshow(red_nuclei_rm);
plt.scatter(maxi_coords_nuclei[1], maxi_coords_nuclei[0]);

In [ ]:
markers_peak_nuclei = ndi.label(local_peak_nuclei)[0]

In [ ]:
labels_nuclei = segmentation.watershed(red_smooth, markers_peak_nuclei, mask=red_nuclei_rm, connectivity=2)
plt.imshow(shuffle_labels(labels_nuclei), cmap='magma');

In [ ]:
# Input image (Red dye channel) and Mask used for segmentation

imgs = [red_smooth, red_nuclei_rm]
T = ["Red channel input image","Red Nuclei (mask)"]
plot_figs(imgs, T)

In [ ]:
# Watershed Segmentation of nuclei of macropahges (Red - CD163), using peak_local_max to generate markers
image_label_overlay = label2rgb(labels_nuclei, image=slide, bg_label=0, alpha=0.5)
plt.imshow(image_label_overlay);

In [ ]:
labels_nuclei_full = segmentation.watershed(red_smooth, markers_peak_nuclei, mask=h_erode, connectivity=2)
plt.imshow(shuffle_labels(labels_nuclei_full), cmap='magma');

In [ ]:
# Watershed Segmentation of nuclei of macropahges (Red - CD163), using peak_local_max to generate markers
# and using the full hematoxylin mask
image_label_overlay2 = label2rgb(labels_nuclei_full, image=slide, bg_label=0, alpha=0.5)
plt.imshow(image_label_overlay);

In [ ]:
labels_nuclei_red = segmentation.watershed(red_smooth, markers_peak_nuclei, mask=red_erode, connectivity=2)
plt.imshow(shuffle_labels(labels_nuclei_red), cmap='magma');

In [ ]:
# Watershed Segmentation of nuclei of macropahges (Red - CD163), using peak_local_max to generate markers
# based on nucleated red cells
image_label_overlay3 = label2rgb(labels_nuclei_red, image=slide, bg_label=0, alpha=0.5)
plt.imshow(image_label_overlay3);

# Fill in nuclei on Red channel
### Erosion on Red channel (red_smooth)
morphology.reconstruction: method = erosion

In [ ]:
# Fill in holes/nuclei
seed = np.copy(red_smooth)
seed[1:-1, 1:-1] = red_smooth.max()
mask = red_smooth

filled = morphology.reconstruction(seed, mask, method='erosion')

plt.imshow(filled);

In [ ]:
# Try all threshold filters
filters.try_all_threshold(filled, figsize=(15, 20));

In [ ]:
# Select Threshold for segmentation:
# Using Yen Threshold as a start for analysis

yen_threshold = filters.threshold_yen(filled)
red_yen = filled > yen_threshold
print("YEN Threshold = ", yen_threshold)

In [ ]:
images_threshold = [slide, filled, red_yen]
titles_threshold = ["Original image", "Red - Erosion/fil holes", "Red - Yen"]

plot_figs(images_threshold, titles_threshold)

In [ ]:
# remove small objects
red_yen_rm = morphology.remove_small_objects(red_yen, min_size=500)
plt.imshow(red_yen_rm);

In [ ]:
red_yen_erode = morphology.binary_erosion(red_yen_rm)
plt.imshow(red_yen_erode);

In [ ]:
red_nuclei2 = (h_erode & red_yen_erode)
plt.imshow(red_nuclei2);

In [ ]:
red_nuclei_rm2 = morphology.remove_small_objects(red_nuclei2, min_size=100)
plt.imshow(red_nuclei_rm2);

In [ ]:
images_nuclei = [slide, h_erode, red_yen_erode, red_nuclei_rm2]
titles_nuclei = ["Original image", "Nuclei", "Red Mask (erosion)", "Red nuclei"]

plot_figs(images_nuclei, titles_nuclei)

## Watershed Segmentation - Red channel (erosion)

In [ ]:
distance_red_nuclei2 = ndi.distance_transform_edt(red_nuclei_rm2) 
#Distance map - distance to travel to background
#Smoother mask
plt.imshow(exposure.adjust_gamma(distance_red_nuclei2, 0.5))
plt.title('Distance to background map: Red nuclei');

In [ ]:
local_peak_nuclei2 = feature.peak_local_max(distance_red_nuclei2, min_distance=25, indices=False)
local_peak_nuclei2.shape

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))

maxi_coords_nuclei2 = np.nonzero(local_peak_nuclei2)
# np.nonzero - Return the indices of the elements that are non-zero.
ax.imshow(red_nuclei_rm2);
plt.scatter(maxi_coords_nuclei2[1], maxi_coords_nuclei2[0]);

In [ ]:
markers_peak_nuclei2 = ndi.label(local_peak_nuclei2)[0]

In [ ]:
labels_nuclei_erosion = segmentation.watershed(red_smooth, markers_peak_nuclei2, mask=red_nuclei_rm2, connectivity=2)
plt.imshow(shuffle_labels(labels_nuclei_erosion), cmap='magma');

In [ ]:
image_label_overlay4 = label2rgb(labels_nuclei_erosion, image=slide, bg_label=0, alpha=0.5)
plt.imshow(image_label_overlay4);

In [ ]:
plot_figs([slide, image_label_overlay4], ["Original", "Segmentation"])

In [ ]:
labels_nuclei_erosion2 = segmentation.watershed(red_smooth, markers_peak_nuclei2, mask=h_erode, connectivity=2)
plt.imshow(shuffle_labels(labels_nuclei_erosion2), cmap='magma');

In [ ]:
image_label_overlay5 = label2rgb(labels_nuclei_erosion2, image=slide, bg_label=0, alpha=0.5)
plt.imshow(image_label_overlay5);

In [ ]:
labels_nuclei_erosion3 = segmentation.watershed(red_smooth, markers_peak_nuclei2, mask=red_yen_erode, connectivity=2)
plt.imshow(shuffle_labels(labels_nuclei_erosion3), cmap='magma');

In [ ]:
image_label_overlay6 = label2rgb(labels_nuclei_erosion3, image=slide, bg_label=0, alpha=0.5)
plt.imshow(image_label_overlay6);

In [ ]:
labels_nuclei_erosion4 = segmentation.watershed(red_yen_erode, markers_peak_nuclei2, mask=red_yen_erode, connectivity=2)
plt.imshow(shuffle_labels(labels_nuclei_erosion4), cmap='magma');

In [ ]:
image_label_overlay7 = label2rgb(labels_nuclei_erosion4, image=slide, bg_label=0, alpha=0.5)
plt.imshow(image_label_overlay7);

In [ ]:
labels_nuclei_erosion5 = segmentation.watershed(red_smooth, markers_peak_nuclei2)
plt.imshow(shuffle_labels(labels_nuclei_erosion5), cmap='magma');

In [ ]:
image_label_overlay8 = label2rgb(labels_nuclei_erosion5, image=slide, bg_label=0, alpha=0.5)
plt.imshow(image_label_overlay8);